This example shows how to use cython with notebooks (part of the code is extracted from the official cython documentation, https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html)

First load the extension cython

In [2]:
%load_ext cython

## Embeeded cython code

Append %%cython at the beginning of the cell to use cython (use --annotate to view C-Python interactions in the code)

In [11]:
%%cython --annotate

# This function returns the first 100 prime numbers and return them as a list
def primes():
    cdef int primes[100]
    cdef int i, j, k
    
    primes[0] = 2
    primes[1] = 3
    i, j = 4, 2
    while j < 100:
        k = 0
        while k < j-1 and i % primes[k] != 0:
            k += 1
        if i % primes[k] != 0:
            primes[j] = i
            j += 1
        i += 1
    
    # Convert C array to python list
    return [p for p in primes]
    

Now execute the function we just defined above

In [12]:
', '.join(map(str, primes()))

'2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541'

As you can see in the yellow lines, the code interacts with Python in when doing a list comprehension and when calling the function. But also "i % primes[k] != 0" is colored with yellow. This is because cython has the behaviour of the Python interpreter by default and it performs runtime checks (in this case it makes sure we dont do a division by zero)

There are multiple ways to disable runtime checks by setting compile directives (https://cython.readthedocs.io/en/latest/src/userguide/source_files_and_compilation.html#compiler-directives).
For instance, we can use decorators:

## Compiler directives

In [24]:
%%cython --annotate

# We need to do this import statement
cimport cython

# Now disable division by zero runtime check
@cython.cdivision(True)
def primes():
    cdef int primes[100]
    cdef int i, j, k
    
    primes[0] = 2
    primes[1] = 3
    i, j = 4, 2
    while j < 100:
        k = 0
        while k < j-1 and i % primes[k] != 0:
            k += 1
        if i % primes[k] != 0:
            primes[j] = i
            j += 1
        i += 1
    
    # Convert C array to python list
    return [p for p in primes]

In [20]:
', '.join(map(str, primes()))

'2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541'

## C++ and Cython

We can get ride of the features that offers C++ inside Cython. For instance, we can use the data structure 'vector'
to represent the primes list in the example above

In [31]:
%%cython --annotate
# distutils: language=c++

cimport cython
from libcpp.vector cimport vector

@cython.cdivision(True)
def primes():
    cdef vector[int] primes
    cdef int i, j, k
    
    primes.reserve(100)
    primes[0] = 2
    primes[1] = 3
    i, j = 4, 2
    while j < 100:
        k = 0
        while k < j-1 and i % primes[k] != 0:
            k += 1
        if i % primes[k] != 0:
            primes.push_back(i)
            j += 1
        i += 1
    
    # vector is converted automatically in a python list
    return primes

In [32]:
', '.join(map(str, primes()))

'5, 7, 8, 9, 12, 19, 22, 26, 31, 33, 34, 39, 46, 51, 58, 61, 67, 69, 73, 74, 82, 86, 87, 94, 103, 106, 109, 111, 118, 121, 123, 127, 129, 131, 137, 139, 141, 142, 143, 149, 151, 157, 158, 159, 163, 166, 167, 169, 173, 177, 178, 179, 181, 187, 191, 193, 194, 197, 199, 202, 211, 213, 214, 221, 223, 226, 227, 229, 233, 237, 239, 241, 249, 251, 253, 257, 263, 267, 269, 271, 277, 281, 283, 289, 291, 293, 299, 303, 307, 311, 313, 317, 319, 321, 331, 337, 339, 347'